Data_Engineering_TIL_(20191024)

#### [학습한 Contents]

- AWS 튜토리얼

1) 주제 : Build a Log Analytics Solution

2) URL : https://aws.amazon.com/ko/getting-started/projects/build-log-analytics-solution


#### [학습목표]


- 데이터 소스에 Kinesis 에이전트를 설정하여 데이터를 수집하고 이를 Amazon Kinesis Firehose로 지속적으로 전송


- Kinesis Firehose를 사용하여 엔드 투 엔드 데이터 전달 스트림을 생성합니다. 전달 스트림은 데이터를 에이전트에서 대상으로 전송

  ** 대상은 Amazon Kinesis Analytics, Amazon Redshift, Amazon Elasticsearch Service 및 Amazon S3 등 가능


- Amazon Kinesis Analytics에서 SQL 쿼리를 사용하여 수신되는 로그 데이터를 처리합니다.


- Kinesis Analytics에서 Amazon Elasticsearch Service로 처리된 데이터를 로드하여 데이터를 인덱싱합니다.


- Kibana를 사용하여 처리된 데이터를 분석 및 시각화합니다.


#### [구현목표 아키텍처]

<img src="1.png" width="1000" />

#### [실습내용 요약]

- Step 1: Set Up Prerequisites


- Step 2: Create an Amazon Kinesis Firehose Delivery Stream


- Step 3: Install and Configure the Amazon Kinesis Agent


- Step 4: Create an Amazon Elasticsearch Service Domain


- Step 5: Create a Second Amazon Kinesis Firehose Delivery Stream


- Step 6: Create an Amazon Kinesis Analytics Application


- Step 7: View the Aggregated Streaming Data


- Step 8: Clean Up

#### [실습 상세내용]

### Step 1: Set Up Prerequisites

step 1-1) ec2 인스턴스 1개 생성

- Amazon Linux 2 AMI (HVM), SSD Volume Type 으로 설치


- 보안그룹은 22번 포트만 열어준다

step 1-2) ec2에 IAM 권한부여

아래 그림과 같이 IAM 롤을 하나 만들어서 ec2에 부여해준다.

<img src="3.png" width="1000" />

step 1-3) ec2 인스턴스로 접속 및 `Fake Apache Log Generator` 설치

1) SSH로 생성한 인스턴스 접속


2) `sudo yum update` 명령어 실행


3) `sudo yum install git` 명령어 실행


4) `git clone https://github.com/kiritbasu/Fake-Apache-Log-Generator.git` 명령어 실행

- 참고 URL : https://github.com/kiritbasu/Fake-Apache-Log-Generator

5) Fake Apache Log Generator 구동을 위한 pip 설치

- 참고 URL : https://docs.aws.amazon.com/ko_kr/elasticbeanstalk/latest/dg/eb-cli3-install-linux.html

5-1) `curl -O https://bootstrap.pypa.io/get-pip.py` 명령어 실행

5-2) `python get-pip.py --user` 명령어 실행

5-3) `ls -a ~` 명령어 실헹

5-4) `vi .bash_profile` 명령어 실행

아래 그림과 같이 수정

<img src="2.png" width="1000" />

5-5) `pip --version` 명령어를 실행하여 pip가 정상적으로 설치되었는지 확인

6) Fake Apache Log Generator 폴더로 이동해서 아래 명령어를 실행한다.

`python -m pip install -r requirements.txt --user`

### Step 2: Create an Amazon Kinesis Firehose Delivery Stream

아래 그림과 같이 kinesis firehose를 생성해준다.

<img src="6.png" width="1000" />

### Step 3: Install and Configure the Amazon Kinesis Agent


step 3-1) `git clone https://github.com/awslabs/amazon-kinesis-agent.git` 명령어 실행

step 3-2) kinesis agent 폴더로 이동해서 `sudo ./setup --install` 명령어 실행

step 3-3) Fake-Apache-Log-Generator폴더가 설치되어 있는 위치로 이동해서 `sudo mv Fake-Apache-Log-Generator /var/log/aws-kinesis-agent/` 명령어를 실행하여 경로를 옮겨준다.

step 3-4) Fake-Apache-Log-Generator폴더내로 이동하여 아래 명령어를 실행하여 백그라운드로 로그생성기를 실행해준다.

`python apache-fake-log-gen.py -n 0 -o LOG &`

step 3-3) 아래 그림과 같이 `sudo vi /etc/aws-kinesis/agent.json` 명령을 실행하여 vim으로 수정해준다.

1) filepattern : 로그파일 경로

   ** 주의사항 : 로그파일 경로는 ec2-user's directory 내에 하지말것, 키네시스 에이전트가 가져올 수 없음
   
`move your logs folder out of ec2-user's home directory.`

`the kinesis agent runs under aws-kinesis-user-agent and cannot access ec2-user's directory.`

`I put my kinesis directory in /var/log/kinesis and ran chown -R aws-kinesis-user-agent:aws-kinesis-user-agent`

2) deliveryStream : 파이어호스 이름

<img src="7.png" width="1000" />

In [ ]:
{   
    "cloudwatch.endpoint": "monitoring.us-west-2.amazonaws.com",   
    "cloudwatch.emitMetrics": true,   
    "firehose.endpoint": "firehose.us-west-2.amazonaws.com",   
    "flows": [     
        {         
            "filePattern": "/var/log/aws-kinesis-agent/Fake-Apache-Log-Generator/access_log_20191025-001937.log",        
            "deliveryStream": "pms-hands-on-firehose",         
            "dataProcessingOptions": [         
                {                 
                    "initialPostion": "START_OF_FILE",                 
                    "maxBufferAgeMillis":"2000",                 
                    "optionName": "LOGTOJSON",                 
                    "logFormat": "COMBINEDAPACHELOG"         
                }]     
        }   
    ] 
} 

step 3-4) 아래 그림과 같이 명령어를 실행하여 키네시스 에이전트를 구동한다.

<img src="8.png" width="1000" />

실행이 성공적으로 수행되면 아래 그림과 같이 `cat /var/log/aws-kinesis-agent/aws-kinesis-agent.log` 명령을 통해 로그가 날아가는 것을 확인할 수 있고, 키네시스 파이어호스와 s3에서도 수신되는 데이터를 확인할 수 있다.

<img src="10.png" width="1000" />

### Step 4: Create an Amazon Elasticsearch Service Domain

- 아래 그림과 같이 생성

<img src="11.png" width="1000" />

### Step 5: Create a Second Amazon Kinesis Firehose Delivery Stream

- 아래 그림과 같이 생성

<img src="12.png" width="1000" />

### Step 6: Create an Amazon Kinesis Analytics Application

- 아래 그림과 같이 키네시스 어넬리틱스를 생성하고 실시간 데이터를 분석해본다.

<img src="13.png" width="1000" />

In [ ]:
CREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" (     
    datetime VARCHAR(30),      
    status INTEGER,      
    statusCount INTEGER); 
 
CREATE OR REPLACE PUMP "STREAM_PUMP" AS      
    INSERT INTO "DESTINATION_SQL_STREAM"         
        SELECT              
            STREAM TIMESTAMP_TO_CHAR('yyyy-MM-dd''T''HH:mm:ss.SSS', LOCALTIMESTAMP) as datetime,              
            "response" as status,              
            COUNT(*) AS statusCount         
        FROM "SOURCE_SQL_STREAM_001"         
        GROUP BY              
            "response",              
            FLOOR(("SOURCE_SQL_STREAM_001".ROWTIME - TIMESTAMP '1970-01-01 00:00:00') minute / 1 TO MINUTE);

그런다음 계속해서 아래 그림과 같이 진행한다.

<img src="14.png" width="1000" />

### Step 7: View the Aggregated Streaming Data

- 일라스틱서치 활용은 아래 그림과 같이 진행한다.

<img src="15.png" width="1000" />

<img src="16.png" width="1000" />

마지막으로 사용한 자원을 잘 종료해준다.